In [2]:
# conda install -c conda-forge openpyxl
import pandas as pd
from openpyxl import load_workbook

EXCEL_FILE = "Feedback_Mid.xlsx"
EXCEL_FILE = "Feedback_End.xlsx"

# %%time
# get meta
df_format = pd.read_excel(EXCEL_FILE, engine="openpyxl")
dimensions = df_format.drop(columns=["Alias", "Naam"]).columns.to_list()
aliases = df_format[["Alias", "Naam"]].dropna().values
aliases = {alias:f"{name} [{alias}]" for alias, name in aliases}
grades = ["ON", "VO", "GO"]

# read the data
df_tot = pd.DataFrame()
for alias in aliases:
    df_alias = (
        pd.read_excel(EXCEL_FILE, sheet_name=alias, engine="openpyxl")
        .assign(By=alias)
        .rename(columns={"Alias": "For"})
        .drop(columns=["Naam"])
        .head(len(aliases))
    )
    df_tot = df_tot.append(df_alias)

# pivot it right
df_tot = (
    df_tot.dropna()
    .melt(id_vars=["For", "By"], var_name="Dimension", value_name="Grade")
    .pivot_table(
        index=["For"],
        columns=["Dimension", "Grade"],
        values="By",
        aggfunc=list,
        dropna=False,
        fill_value=" ",
    )
)

# make it pretty
df_out = df_tot.T.reset_index(["Dimension", "Grade"]).assign(
    Dimension=lambda x: pd.Categorical(x["Dimension"], categories=dimensions, ordered=True),
    Grade=lambda x: pd.Categorical(x["Grade"], categories=grades, ordered=True),
).sort_values(by=["Dimension", "Grade"]).set_index(["Dimension", "Grade"]).T.loc[aliases].rename(aliases)

# save it on a new sheet
book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine = 'openpyxl')
writer.book = book

df_out.to_excel(writer, sheet_name = 'Totaal')
writer.save()
writer.close()

## experiment with CategoricalIndex

In [3]:
def make_CategoricalIndex(index, categories, ordered=True):
    if isinstance(index, pd.MultiIndex):
        index_levels = [
            pd.CategoricalIndex(i, categories=c, ordered=o) if c else i
            for i, c, o in zip(index.levels, categories, ordered)
        ]
        new_index = index.set_levels(index_levels)
    elif isinstance(index, pd.Index):
        new_index = pd.CategoricalIndex(index, categories=categories, ordered=ordered)
    return new_index

In [4]:
df_tot.columns = make_CategoricalIndex(df_tot.columns, categories=[dimensions, grades], ordered=[True, True])
df_tot.index = make_CategoricalIndex(df_tot.index, categories=aliases, ordered=True)

In [5]:
df_tot.sort_index(axis=0)

Dimension         communication           honouring_agreements        \
Grade                        GO ON    VO                    GO ON VO   
For                                                                    
FN         [FN, LF, MD, LC, BV]           [FN, LF, MD, LC, BV]         
LF             [FN, MD, LC, BV]     [LF]  [FN, LF, MD, LC, BV]         
MD         [FN, LF, MD, LC, BV]           [FN, LF, MD, LC, BV]         
LC         [FN, LF, MD, LC, BV]           [FN, LF, MD, LC, BV]         
BV         [FN, LF, MD, LC, BV]           [FN, LF, MD, LC, BV]         

Dimension initiative_contribution                        motivation        \
Grade                          GO    ON    VO                    GO ON VO   
For                                                                         
FN               [LF, MD, LC, BV]        [FN]  [FN, LF, MD, LC, BV]         
LF           [FN, LF, MD, LC, BV]              [FN, LF, MD, LC, BV]         
MD           [FN, LF, MD, LC, BV]              [FN, LF, MD, LC, BV]         
LC               [FN, LF, MD, BV]  [LC]        [FN, LF, MD, LC, BV]         
BV               [FN, LF, MD, LC]        [BV]  [FN, LF, MD, LC, BV]         

Dimension          quality_work               tranceiving_feedback           
Grade                        GO ON        VO                    GO ON    VO  
For                                                                          
FN         [FN, LF, MD, LC, BV]               [FN, LF, MD, LC, BV]           
LF         [FN, LF, MD, LC, BV]               [FN, LF, MD, LC, BV]           
MD         [FN, LF, MD, LC, BV]                   [FN, LF, LC, BV]     [MD]  
LC                 [FN, LF, BV]     [MD, LC]      [FN, LF, LC, BV]     [MD]  
BV         [FN, LF, MD, LC, BV]                   [FN, LF, MD, LC]     [BV]